## Given Reg_ID Finding out Top possible Match Likely to be Colluders

In [ ]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
import string
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score,classification_report,auc,roc_curve

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.utils import resample

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import NearMiss
from imblearn.combine import SMOTETomek
from catboost import CatBoostClassifier
from sklearn import tree

In [ ]:
Feature_df = pd.read_pickle('Feature_df.pkl')
Feature_df.tail(2)

In [ ]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep]

def plot_confusion_mat(y_test, y_pred):
    labels = [0, 1]
    cm = confusion_matrix(y_test, y_pred, labels)
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

    # labels, title and ticks
    ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(['0', '1']); ax.yaxis.set_ticklabels(['0', '1']);

from math import sin, cos, sqrt, atan2, radians

def get_distance_between_locs(p1_lat, p1_lon, p2_lat, p2_lon):
    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(p1_lat)
    lon1 = radians(p1_lon)
    lat2 = radians(p2_lat)
    lon2 = radians(p2_lon)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance
# Manufacture details handled inphone , separated by "," , separated by " "
def get_MANUFACTURER(device_info):
    S =  'Device Name'
    T = 'MANUFACTURER'
    data = [k for k in device_info.split(':')]
    for t in range(len(data)):
        if (T in data[t]) or (S in data[t]):
            s = re.split(' Device OS| BRAND',data[t+1])[0]
            s = s.split(',',1)[0]
            if s[0] == ' ' or s[-1]== ' ':
                s = s.replace(' ','',1)
            return s

def get_MODEL(device_info):
    S = 'Device Modal'
    T = 'MODEL'
    data = [k for k in device_info.split(':')]
    for t in range(len(data)):
        if (T in data[t]) or (S in data[t]):
            s = re.split(' Device OS| BRAND',data[t+1])[0]
            s = s.split(',',1)[0]
            if s[0] == ' ' or s[-1]== ' ':
                s = s.replace(' ','',1)
            return s
def contact_overlap(p1,p2):
    p1_ = set(p1) 
    p2_ = set(p2)
    if len(p1_.union(p2_)) == 0:
        k = 0
    else:
        k = ( len(p1_.intersection(p2_))/len(p1_.union(p2_)) ) * 100
    return k
def app_overlap(p1,p2):
    p1_ = set(p1) 
    p2_ = set(p2)
    if len(p1_.union(p2_)) ==0:
        k=0
    else:
        k =( len(p1_.intersection(p2_))/len(p1_.union(p2_)) ) * 100
    return k
def levenshtein_distance(a,b):
    n, m = len(a), len(b)
    if n > m:
        # Make sure n <= m, to use O(min(n,m)) space
        a,b = b,a
        n,m = m,n

    current = range(n+1)
    for i in range(1,m+1):
        previous, current = current, [i]+[0]*n
        for j in range(1,n+1):
            add, delete = previous[j]+1, current[j-1]+1
            change = previous[j-1]
            if a[j-1] != b[i-1]:
                change = change + 1
            current[j] = min(add, delete, change)

    return current[n]
def similarity(A,B):    
    if __name__=="__main__":
        
        k = levenshtein_distance(A,B)  #  k is the similarity coeficients between these two strings A,B
        if (len(A)+len(B))==0:
            k_normalise = float('NaN')
        else:
            k_normalise = k/(len(A)+len(B))
        return k_normalise

def get_corresponding_feature(ID_df):
    
    reg_id   = [ID_df.p1.iloc[0]] 
    given_id = ID_df.p2.unique().tolist()
    All_id   = given_id+reg_id
    
    import pymssql
    cursor = pymssql.connect(server='ec2-15-206-55-153.ap-south-1.compute.amazonaws.com', user='internUser', password='WinZO@123', database='master')
    
    given_id_str = ','.join(str(u) for u in All_id)

    # blocked_users_query = """
    # SELECT TRID,REGISTRATIONID,BLOCK_REASON ,REASON_TEXT ,CREATED_DATE
    # FROM TicTokdb.dbo.STBL_USER_BLOCK
    # where REGISTRATIONID in ({})
    # """.format(given_id_str)
    # blocked_users_df_temp = pd.read_sql(blocked_users_query, cursor)


    user_reg_query = """
    SELECT REGISTRATIONID,NAME,REGISTRATION_DATE,DEVICE_INFO,STATUS from MongoDB.dbo.stbl_Registration
    where REGISTRATIONID in ({})
    """.format(given_id_str)
    user_reg_df_temp = pd.read_sql(user_reg_query, cursor)

    user_contacts_query = """
    select TRID,REGISTRATIONID,NAME,CREATED_DATE from MongoDB.DBO.STBL_PHONE_CONTACTS 
    where REGISTRATIONID in ({})
    """.format(given_id_str)
    user_contacts_df_temp = pd.read_sql(user_contacts_query, cursor)


    user_loc_query = """
    SELECT REGISTRATIONID,LATITUDE,LONGITUDE from TicTokdb.dbo.STBL_USER_LOCATION
    where REGISTRATIONID in ({})
    """.format(given_id_str)
    user_loc_df_temp = pd.read_sql(user_loc_query, cursor)

    user_apps_query = """
    SELECT REGISTRATIONID,APP_NAME_ID from TicTokdb.dbo.STBL_USER_APP_NAMES
    where REGISTRATIONID in ({})
    """.format(given_id_str)
    #     print(user_apps_query)
    user_apps_df_temp = pd.read_sql(user_apps_query, cursor)


    user_Rake_query = """
    SELECT CATEGORY,REGISTRATIONID,RAKE from MongoDB.analysis.DAILY_FORMATWISE_RAKE
    where REGISTRATIONID in ({})
    """.format(given_id_str)
    user_Rake_df_temp = pd.read_sql(user_Rake_query, cursor)
    user_Rake_df_temp = user_Rake_df_temp.loc[user_Rake_df_temp.groupby('REGISTRATIONID').RAKE.idxmax()].reset_index(drop=True)


    Add_Cash_query = """
    SELECT REGISTRATIONID,PAYAMOUNT,PAYDATE,PAYSTATUS,CREATED_DATE FROM TicTokdb.dbo.STBL_CASH_IN_WALLET
    where REGISTRATIONID in ({}) AND PAYSTATUS = 'SUCCESS'
    """.format(given_id_str)
    Add_Cash_df_temp = pd.read_sql(Add_Cash_query, cursor)
    Add_Cash_df_temp = Add_Cash_df_temp.loc[Add_Cash_df_temp.groupby('REGISTRATIONID').PAYDATE.idxmin()].reset_index(drop=True)

    wb_bazzi_trans_query = """
    SELECT REGISTRATIONID,CHALLENGE_ID,AMOUNT,TRANS_TYPE FROM TicTokdb.dbo.STBL_WB_BAAZI_TRANSACTION swbt
    where REGISTRATIONID in ({})
    """.format(given_id_str)
    user_commongames_df_temp = pd.read_sql(wb_bazzi_trans_query, cursor)
#################################

    user_loc_df_temp.set_index(['REGISTRATIONID'],inplace=True)
    user_apps_df_temp.set_index(['REGISTRATIONID'],inplace=True)
    Add_Cash_df_temp.set_index(['REGISTRATIONID'],inplace=True)
    user_Rake_df_temp.set_index(['REGISTRATIONID'],inplace=True)
    user_commongames_df_temp.set_index(['REGISTRATIONID'],inplace=True)
#     blocked_users_df_temp.set_index(['REGISTRATIONID'],inplace=True)
    
    user_reg_df_temp.set_index(['REGISTRATIONID'],inplace=True)    
    user_contacts_df_temp.set_index(['REGISTRATIONID'],inplace=True)

    credit_df_temp = user_commongames_df_temp[user_commongames_df_temp.TRANS_TYPE == 'C']
    debit_df_temp  = user_commongames_df_temp[user_commongames_df_temp.TRANS_TYPE == 'D']

#########################################
    block_df_temp_list = []
    count = 0
#     Name_similarity,matching_contact_count = 0,0
    user = reg_id[0]
    if count%25 == 0:
        print(user,count)
    count+=1

    p1_location = user_loc_df_temp[user_loc_df_temp.index==user]
    if p1_location.shape[0] > 0:
        p1_lat, p1_lon = p1_location['LATITUDE'].iloc[0], p1_location['LONGITUDE'].iloc[0]
    else:
        p1_lat, p1_lon = 0, 0
    try:
        Rake_value_p1 = user_Rake_df_temp[user_Rake_df_temp.index==user]['RAKE'].iloc[0]
    except:
        Rake_value_p1 = np.nan
    try:
        Frst_AddCash_p1 = Add_Cash_df_temp[Add_Cash_df_temp.index==user]['PAYAMOUNT'].iloc[0]
    except:
        Frst_AddCash_p1  = 0
# ###        
    try:
        p1_name   = user_reg_df_temp[user_reg_df_temp.index==user]['NAME'].iloc[0]
    except:
        p1_name = ''
    p1_contact  =  user_contacts_df_temp[user_contacts_df_temp.index==user]['NAME']
# ###
    p1_app    = user_apps_df_temp[user_apps_df_temp.index==user]['APP_NAME_ID']
    p1_game   = set(user_commongames_df_temp[user_commongames_df_temp.index==user]['CHALLENGE_ID'])

#############################    
    for user2 in given_id:
        if (user != user2):

            p2_location = user_loc_df_temp[user_loc_df_temp.index==user2]
            if p2_location.shape[0] > 0:
                p2_lat, p2_lon = p2_location['LATITUDE'].iloc[0], p2_location['LONGITUDE'].iloc[0]
            else:
                p2_lat, p2_lon = 0, 0 

            p1_p2_distance = np.nan
            if p1_location.shape[0] > 0 and p2_location.shape[0] > 0:
                p1_p2_distance = get_distance_between_locs(p1_lat, p1_lon, p2_lat, p2_lon)   ###
###                    
            try:
                p2_name   = user_reg_df_temp[user_reg_df_temp.index==user2]['NAME'].iloc[0]
            except:
                p2_name = ''
                
            Name_similarity = similarity(p1_name,p2_name)
            p2_contact  =  user_contacts_df_temp[user_contacts_df_temp.index==user2]['NAME']
            matching_contact_count = contact_overlap(p1_contact,p2_contact)
###
            try:
                Rake2 = user_Rake_df_temp[user_Rake_df_temp.index==user2]['RAKE'].iloc[0]
            except:
                Rake2 = np.nan
            try:
                Frst_AddCash2 = Add_Cash_df_temp[Add_Cash_df_temp.index==user2]['PAYAMOUNT'].iloc[0]
            except:
                Frst_AddCash2  = 0
            p2_app     = user_apps_df_temp[user_apps_df_temp.index==user2]['APP_NAME_ID']
            app_match  = app_overlap(p1_app,p2_app)

            p2_game = set(user_commongames_df_temp[user_commongames_df_temp.index==user2]['CHALLENGE_ID'])
            common_games = p1_game.intersection(p2_game)

            mean_Rake = (Rake_value_p1+Rake2)/2
            mean_frst_addcash = (Frst_AddCash_p1+Frst_AddCash2)/2

            if len(common_games) != 0 :

                common_games_percent_of_A_games = (len(common_games)/len(p1_game) )*100                    
                common_games_percent_of_B_games = ( len(common_games)/len(p2_game) )*100
                max_common_gameAB__of_A_B   = max(common_games_percent_of_A_games,common_games_percent_of_B_games)

                # common_games_A_B_profit   && # common_games_A_B_c_by_d
                com_game_credit_A = credit_df_temp[(credit_df_temp.index == user) & (credit_df_temp.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()
                com_game_debit_A  = debit_df_temp[(debit_df_temp.index == user) & (debit_df_temp.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()
                com_game_credit_B = credit_df_temp[(credit_df_temp.index == user2) & (credit_df_temp.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()
                com_game_debit_B  = debit_df_temp[(debit_df_temp.index == user2) & (debit_df_temp.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()

                if ((com_game_debit_A+com_game_debit_B) != 0):
#                     A_B_Profit = ((com_game_credit_A+com_game_credit_B) - (com_game_debit_A+com_game_debit_B))/(com_game_debit_A+com_game_debit_B)
                    A_B_C_BY_D = (com_game_credit_A+com_game_credit_B)/(com_game_debit_A+com_game_debit_B)
                else:
#                     A_B_Profit =  np.nan
                    A_B_C_BY_D =  np.nan

                block_df_temp_list.append({
                        'P1_regID':user,
                        'P2_regID': user2,
                        'Name_similarity': Name_similarity,
                        'Contacts_Overlap_%':matching_contact_count,
                        'p1_p2_distance_KM' : p1_p2_distance,
                        'Apps_overlap_%': app_match,
                        'A_B_C_BY_D':A_B_C_BY_D,
                        'mean_Rake' : mean_Rake,
                        'mean_Frst_AddCash_1&2': mean_frst_addcash,
                        'max_common_gameAB_%_of_A&B' : max_common_gameAB__of_A_B,
#                         'A_B_Profit':A_B_Profit,



                    })
            else:
                block_df_temp_list.append({
                    'P1_regID':user,
                    'P2_regID': user2,
                    'Name_similarity': Name_similarity,
                    'Contacts_Overlap_%':matching_contact_count,
                    'p1_p2_distance_KM' : p1_p2_distance,
                    'Apps_overlap_%': app_match,
                    'mean_Rake' : mean_Rake,
                    'mean_Frst_AddCash_1&2': mean_frst_addcash
                })

    #     print('About to Complete')
    Test_df = pd.DataFrame(block_df_temp_list)
    return Test_df

def get_one_week_feature(ID_df):
    given_id2 = ID_df.p2.tolist()
    given_id1 = ID_df.p1.tolist()
    given_id = given_id1 + given_id2
    given_id = set(given_id)
    given_id = list(given_id)
    import pymssql
    cursor = pymssql.connect(server='ec2-15-206-55-153.ap-south-1.compute.amazonaws.com', user='internUser', password='WinZO@123', database='master')

    given_id_str = ','.join(str(u) for u in given_id)

    # blocked_users_query = """
    # SELECT TRID,REGISTRATIONID,BLOCK_REASON ,REASON_TEXT ,CREATED_DATE
    # FROM TicTokdb.dbo.STBL_USER_BLOCK
    # where REGISTRATIONID in ({})
    # """.format(given_id_str)
    # blocked_users_df_temp = pd.read_sql(blocked_users_query, cursor)


    user_reg_query = """
    SELECT REGISTRATIONID,NAME,REGISTRATION_DATE,DEVICE_INFO,STATUS from MongoDB.dbo.stbl_Registration
    where REGISTRATIONID in ({})
    """.format(given_id_str)
    user_reg_df_temp = pd.read_sql(user_reg_query, cursor)
    
    user_contacts_query = """
    select TRID,REGISTRATIONID,NAME,CREATED_DATE from MongoDB.DBO.STBL_PHONE_CONTACTS 
    where REGISTRATIONID in ({})
    """.format(given_id_str)
    user_contacts_df_temp = pd.read_sql(user_contacts_query, cursor)


    user_loc_query = """
    SELECT REGISTRATIONID,LATITUDE,LONGITUDE from TicTokdb.dbo.STBL_USER_LOCATION
    where REGISTRATIONID in ({})
    """.format(given_id_str)
    user_loc_df_temp = pd.read_sql(user_loc_query, cursor)

    user_apps_query = """
    SELECT REGISTRATIONID,APP_NAME_ID from TicTokdb.dbo.STBL_USER_APP_NAMES
    where REGISTRATIONID in ({})
    """.format(given_id_str)
    #     print(user_apps_query)
    user_apps_df_temp = pd.read_sql(user_apps_query, cursor)


    user_Rake_query = """
    SELECT CATEGORY,REGISTRATIONID,RAKE from MongoDB.analysis.DAILY_FORMATWISE_RAKE
    where REGISTRATIONID in ({})
    """.format(given_id_str)
    user_Rake_df_temp = pd.read_sql(user_Rake_query, cursor)
    user_Rake_df_temp = user_Rake_df_temp.loc[user_Rake_df_temp.groupby('REGISTRATIONID').RAKE.idxmax()].reset_index(drop=True)


    Add_Cash_query = """
    SELECT REGISTRATIONID,PAYAMOUNT,PAYDATE,PAYSTATUS,CREATED_DATE FROM TicTokdb.dbo.STBL_CASH_IN_WALLET
    where REGISTRATIONID in ({}) AND PAYSTATUS = 'SUCCESS'
    """.format(given_id_str)
    Add_Cash_df_temp = pd.read_sql(Add_Cash_query, cursor)
    Add_Cash_df_temp = Add_Cash_df_temp.loc[Add_Cash_df_temp.groupby('REGISTRATIONID').PAYDATE.idxmin()].reset_index(drop=True)

    wb_bazzi_trans_query = """
    SELECT REGISTRATIONID,CHALLENGE_ID,AMOUNT,TRANS_TYPE FROM TicTokdb.dbo.STBL_WB_BAAZI_TRANSACTION swbt
    where REGISTRATIONID in ({})
    """.format(given_id_str)
    user_commongames_df_temp = pd.read_sql(wb_bazzi_trans_query, cursor)
    #################################

    user_loc_df_temp.set_index(['REGISTRATIONID'],inplace=True)
    user_apps_df_temp.set_index(['REGISTRATIONID'],inplace=True)
    Add_Cash_df_temp.set_index(['REGISTRATIONID'],inplace=True)
    user_Rake_df_temp.set_index(['REGISTRATIONID'],inplace=True)
    user_commongames_df_temp.set_index(['REGISTRATIONID'],inplace=True)
    user_reg_df_temp.set_index(['REGISTRATIONID'],inplace=True)    
    user_contacts_df_temp.set_index(['REGISTRATIONID'],inplace=True)
    
    #     blocked_users_df_temp.set_index(['REGISTRATIONID'],inplace=True)
    

    credit_df_temp = user_commongames_df_temp[user_commongames_df_temp.TRANS_TYPE == 'C']
    debit_df_temp  = user_commongames_df_temp[user_commongames_df_temp.TRANS_TYPE == 'D']

    #########################################
#     user_already_processed = set()
    block_df_temp_list = []
    count = 0
    i=0
    Name_similarity,matching_contact_count = 0,0
    for i in range(len(given_id1)):
        user = given_id1[i]
        user2 = given_id2[i]
        if count%100 == 0:
            print(user,count)
        count+=1

        p1_location = user_loc_df_temp[user_loc_df_temp.index==user]
        if p1_location.shape[0] > 0:
            p1_lat, p1_lon = p1_location['LATITUDE'].iloc[0], p1_location['LONGITUDE'].iloc[0]
        else:
            p1_lat, p1_lon = 0, 0
        try:
            Rake_value_p1 = user_Rake_df_temp[user_Rake_df_temp.index==user]['RAKE'].iloc[0]
        except:
            Rake_value_p1 = 0
        try:
            Frst_AddCash_p1 = Add_Cash_df_temp[Add_Cash_df_temp.index==user]['PAYAMOUNT'].iloc[0]
        except:
            Frst_AddCash_p1  = 0
    #        
        try:
            p1_name   = user_reg_df_temp[user_reg_df_temp.index==user]['NAME'].iloc[0]
        except:
            p1_name = ''
        p1_contact  =  user_contacts_df_temp[user_contacts_df_temp.index==user]['NAME']
    
        p1_app    = user_apps_df_temp[user_apps_df_temp.index==user]['APP_NAME_ID']
        p1_game   = set(user_commongames_df_temp[user_commongames_df_temp.index==user]['CHALLENGE_ID'])

    #############################    
    # for user2 in given_id2:
    # if ((user2 not in user_already_processed)&(user != user2)&(given_id1[i]==user)&(given_id2[i]==user2)):
    #     i+=1
        p2_location = user_loc_df_temp[user_loc_df_temp.index==user2]
        if p2_location.shape[0] > 0:
            p2_lat, p2_lon = p2_location['LATITUDE'].iloc[0], p2_location['LONGITUDE'].iloc[0]
        else:
            p2_lat, p2_lon = 0, 0 

        p1_p2_distance = np.nan
        if p1_location.shape[0] > 0 and p2_location.shape[0] > 0:
            p1_p2_distance = get_distance_between_locs(p1_lat, p1_lon, p2_lat, p2_lon)   ###
    ###                    
        try:
            p2_name   = user_reg_df_temp[user_reg_df_temp.index==user2]['NAME'].iloc[0]
        except:
            p2_name = ''
        Name_similarity = similarity(p1_name,p2_name)
        p2_contact  =  user_contacts_df_temp[user_contacts_df_temp.index==user2]['NAME']
        matching_contact_count = contact_overlap(p1_contact,p2_contact)
    ###
        try:
            Rake2 = user_Rake_df_temp[user_Rake_df_temp.index==user2]['RAKE'].iloc[0]
        except:
            Rake2 = 0
        try:
            Frst_AddCash2 = Add_Cash_df_temp[Add_Cash_df_temp.index==user2]['PAYAMOUNT'].iloc[0]
        except:
            Frst_AddCash2  = 0
        p2_app     = user_apps_df_temp[user_apps_df_temp.index==user2]['APP_NAME_ID']
        app_match  = app_overlap(p1_app,p2_app)

        p2_game = set(user_commongames_df_temp[user_commongames_df_temp.index==user2]['CHALLENGE_ID'])
        common_games = p1_game.intersection(p2_game)

        mean_Rake = (Rake_value_p1+Rake2)/2
        mean_frst_addcash = (Frst_AddCash_p1+Frst_AddCash2)/2

        if len(common_games) != 0 :

            common_games_percent_of_A_games = (len(common_games)/len(p1_game) )*100                    
            common_games_percent_of_B_games = ( len(common_games)/len(p2_game) )*100
            max_common_gameAB__of_A_B   = max(common_games_percent_of_A_games,common_games_percent_of_B_games)

        # common_games_A_B_profit   && # common_games_A_B_c_by_d
            com_game_credit_A = credit_df_temp[(credit_df_temp.index == user) & (credit_df_temp.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()
            com_game_debit_A  = debit_df_temp[(debit_df_temp.index == user) & (debit_df_temp.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()
            com_game_credit_B = credit_df_temp[(credit_df_temp.index == user2) & (credit_df_temp.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()
            com_game_debit_B  = debit_df_temp[(debit_df_temp.index == user2) & (debit_df_temp.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()

            if ((com_game_debit_A+com_game_debit_B) != 0):
                A_B_Profit = ((com_game_credit_A+com_game_credit_B) - (com_game_debit_A+com_game_debit_B))/(com_game_debit_A+com_game_debit_B)
                A_B_C_BY_D = (com_game_credit_A+com_game_credit_B)/(com_game_debit_A+com_game_debit_B)
            else:
                A_B_Profit =  np.nan
                A_B_C_BY_D =  np.nan

            block_df_temp_list.append({
                    'P1_regID':user,
                    'P2_regID': user2,
                    'Name_similarity': Name_similarity,
                    'Contacts_Overlap_%':matching_contact_count,
                    'p1_p2_distance_KM' : p1_p2_distance,
                    'Apps_overlap_%': app_match,
                    'A_B_C_BY_D':A_B_C_BY_D,
                    'mean_Rake' : mean_Rake,
                    'mean_Frst_AddCash_1&2': mean_frst_addcash,
                    'max_common_gameAB_%_of_A&B' : max_common_gameAB__of_A_B,
    #                         'A_B_Profit':A_B_Profit,



                })
        else:
            block_df_temp_list.append({
                'P1_regID':user,
                'P2_regID': user2,
                'Name_similarity': Name_similarity,
                'Contacts_Overlap_%':matching_contact_count,
                'p1_p2_distance_KM' : p1_p2_distance,
                'Apps_overlap_%': app_match,
                'mean_Rake' : mean_Rake,
                'mean_Frst_AddCash_1&2': mean_frst_addcash
            })

#     user_already_processed.add(user)
    Test_df = pd.DataFrame(block_df_temp_list)
    return Test_df
# Test_df.head()

def Top_p2(Xtest,classifier_r):
    df_ = Xtest.drop(['P1_regID','P2_regID'],axis='columns')
    predictions = classifier_r.predict_proba(df_)
#     print(predictions)
    pd_df = pd.DataFrame(predictions, columns=["A", "B"])
    pd_df.drop(pd_df[pd_df['B'] < 0.75].index, inplace = True)    #### Not Needed 
    pd_df.sort_values("B", axis = 0, ascending = False,inplace=True)
    
    out_df = pd_df.head(5)
    out_index = out_df.index
    return pd_df,Xtest.iloc[out_index]


#### Replacing Nan Values

In [ ]:
df = Feature_df.copy()
df.replace([np.inf, -np.inf], np.nan, inplace=True)

df["mean_Rake"] = df[["Rake", "Rake2"]].mean(axis=1)
df["mean_Frst_AddCash_1&2"] = df[["Frst_AddCash", "Frst_AddCash2"]].mean(axis=1)
df["mean_Frst_AddCash_1&2"].fillna(value=0, inplace=True)
df["max_common_gameAB_%_of_A&B"] = df[["common_gameAB_%_of_A","common_gameAB_%_of_B"]].max(axis=1)
df['max_common_gameAB_%_of_A&B'].fillna(value=0, inplace=True)
df['A_B_C_BY_D'].fillna(value=0, inplace=True)
df['Name_similarity'].fillna(value=df["Name_similarity"].max(), inplace=True)


In [ ]:
df.drop(['P1_regID', 'P2_regID','p1_name','p2_name','Diff_p1block_p2_reg_minutes','Rake','Category','Rake2',
       'Category2','common_games','Frst_AddCash', 'Frst_AddCash2','common_gameAB_%_of_A', 'common_gameAB_%_of_B',
       'A_B_Profit'],axis='columns',inplace=True)


In [ ]:
df.isnull().sum()

## Sampling and Splitting the Data
#### Random Undersampling and Upsamling using Smote

In [ ]:
%%time

count_class_0, count_class_1 = df.Class.value_counts()
# Divide by class
df_class_0 = df[df['Class'] == 0]
df_class_1 = df[df['Class'] == 1]

# Undersample 0-class and concat the DataFrames of both class
df_class_0_under = df_class_0.sample(2000)
# df_class_1       = df_class_1.sample(1000,replace=True)
df_test_ = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-over-sampling:')
print(df_test_.Class.value_counts())

y = df_test_.Class
X = df_test_.drop(['Class'], axis=1)
# print(y.value_counts())

from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)
print(y_sm.value_counts())

X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)
# Number of classes in training Data
print(y_train.value_counts())

### Random Forest

In [ ]:
classifier_r = RandomForestClassifier(max_depth = 4,n_estimators = 10, criterion = 'entropy', random_state = 15,max_features = "auto")
classifier_r.fit(X_train, y_train)

# Predicting the Test set results
y_pred_1 = classifier_r.predict(X_train)
print('Training Accuracy :',accuracy_score(y_train,y_pred_1))
y_pred  = classifier_r.predict(X_test)
print('Testing Accuracy :',accuracy_score(y_test,y_pred))

# Making the Confusion Matrix
print("Classification Report[Randomforest]: \n", classification_report(y_test, y_pred))
print("classifier_r.feature_importances_[Randomforest]: \n", classifier_r.feature_importances_)


feature_importance_r = classifier_r.feature_importances_
col = X_train.columns.values.tolist()
feature_df_random = pd.DataFrame(list(zip(col, feature_importance_r)),
               columns =['Features', 'Feature_weight'])

feature_df_random.sort_values('Feature_weight',ascending=False)

In [ ]:
fn = list(X_train.columns)
cn =['0','1']

plt.figure(figsize=(25,5))
tree.plot_tree(classifier_r.estimators_[0],feature_names = fn, 
               class_names=cn,filled=True)

print(tree.export_text(classifier_r.estimators_[0]))

### In one week possible colluders

In [ ]:
%%time

import pandas as pd
import pymssql
cursor = pymssql.connect(server='ec2-15-206-55-153.ap-south-1.compute.amazonaws.com', user='internUser', password='WinZO@123', database='master')

query= '''
select * from (
				SELECT wb1.REGISTRATIONID as p1, wb2.REGISTRATIONID as p2, 
				count(*) as overlap_count
				from
				( 
				SELECT REGISTRATIONID, CHALLENGE_ID from TicTokdb.dbo.STBL_WB_BAAZI_TRANSACTION swbt 
				where GAME_TYPE = 49 AND CREATED_DATE > '2021-08-01 00:00:00'
				) as wb1
				left join
				( 
				SELECT REGISTRATIONID, CHALLENGE_ID from TicTokdb.dbo.STBL_WB_BAAZI_TRANSACTION swbt 
                 where GAME_TYPE = 49 AND CREATED_DATE > '2021-08-01 00:00:00'
				) as wb2
				on (wb1.REGISTRATIONID != wb2.REGISTRATIONID) and (wb1.CHALLENGE_ID = wb2.CHALLENGE_ID)
				GROUP BY wb1.REGISTRATIONID, wb2.REGISTRATIONID
			  ) as wb3
where overlap_count> 20;
'''#.format(temp_str)
ID_df = pd.read_sql(query, cursor)
ID_df.dropna(axis=0,inplace=True)
ID_df.reset_index(drop=True,inplace=True)
print(ID_df.shape)
ID_df.head(2)

In [ ]:
%%time
Test_df = get_one_week_feature(ID_df)

Test_df_copy = Test_df.copy()   ## In case need to save Test_df
Test_df.dropna(axis=0,inplace=True)  ## Just for Testing You Can check which all rows are Nan and replace them accordingly
print(Test_df.shape)

prediction_df,Possible_block_ID_df = Top_p2(Test_df,classifier_r)
print(prediction_df[prediction_df.B > 0.95].shape)
Possible_block_ID_df.head()

### Get Top_p2 corresponding to given reg_Id

In [ ]:
%%time
reg_id = [17772895]
temp_str = ','.join(str(u) for u in reg_id)

import pandas as pd
import pymssql
cursor = pymssql.connect(server='ec2-15-206-55-153.ap-south-1.compute.amazonaws.com', user='internUser', password='WinZO@123', database='master')

query= '''
select * from ( 
SELECT wb1.REGISTRATIONID as p1, wb2.REGISTRATIONID as p2, 
count(*) as overlap_count
from ( 
SELECT REGISTRATIONID, CHALLENGE_ID from TicTokdb.dbo.STBL_WB_BAAZI_TRANSACTION swbt
where GAME_TYPE = 49 and  REGISTRATIONID in ({})
) as wb1
left join ( 
SELECT REGISTRATIONID, CHALLENGE_ID from TicTokdb.dbo.STBL_WB_BAAZI_TRANSACTION swbt
where GAME_TYPE = 49
) as wb2
on (wb1.REGISTRATIONID != wb2.REGISTRATIONID) and (wb1.CHALLENGE_ID = wb2.CHALLENGE_ID)
GROUP BY wb1.REGISTRATIONID, wb2.REGISTRATIONID) as wb3
where overlap_count>10;
'''.format(temp_str)
ID_df_1 = pd.read_sql(query, cursor)
ID_df_1.dropna(axis=0,inplace=True)   ## Just for Testing You Can check which all rows are Nan and replace them accordingly
ID_df_1.reset_index(drop=True,inplace=True)
print(ID_df_1.shape)
ID_df_1.head(2)

In [ ]:
%%time

Test_df_1 = get_corresponding_feature(ID_df_1)

Test_df.dropna(axis=0,inplace=True)  ## Just for Testing You Can check which all rows are Nan and replace them accordingly
Test_df_1_copy = Test_df_1.copy()
print(Test_df_1.shape)

prediction_df_,Top_p2_df = Top_p2(Test_df_1,classifier_r)
print(prediction_df_[prediction_df_.B>0.9].shape)
Top_p2_df.head()

In [ ]:
reg_id   = [ID_df_1.p1.iloc[0]] 
given_id = ID_df_1.p2.unique().tolist()
All_id   = given_id+reg_id

import pymssql
cursor = pymssql.connect(server='ec2-15-206-55-153.ap-south-1.compute.amazonaws.com', user='internUser', password='WinZO@123', database='master')

given_id_str = ','.join(str(u) for u in All_id)

# blocked_users_query = """
# SELECT TRID,REGISTRATIONID,BLOCK_REASON ,REASON_TEXT ,CREATED_DATE
# FROM TicTokdb.dbo.STBL_USER_BLOCK
# where REGISTRATIONID in ({})
# """.format(given_id_str)
# blocked_users_df_temp = pd.read_sql(blocked_users_query, cursor)


user_reg_query = """
SELECT REGISTRATIONID,NAME,REGISTRATION_DATE,DEVICE_INFO,STATUS from MongoDB.dbo.stbl_Registration
where REGISTRATIONID in ({})
""".format(given_id_str)
user_reg_df_temp = pd.read_sql(user_reg_query, cursor)
user_contacts_query = """
select TRID,REGISTRATIONID,NAME,CREATED_DATE from MongoDB.DBO.STBL_PHONE_CONTACTS 
where REGISTRATIONID in ({})
""".format(given_id_str)
user_contacts_df_temp = pd.read_sql(user_contacts_query, cursor)


user_loc_query = """
SELECT REGISTRATIONID,LATITUDE,LONGITUDE from TicTokdb.dbo.STBL_USER_LOCATION
where REGISTRATIONID in ({})
""".format(given_id_str)
user_loc_df_temp = pd.read_sql(user_loc_query, cursor)

user_apps_query = """
SELECT REGISTRATIONID,APP_NAME_ID from TicTokdb.dbo.STBL_USER_APP_NAMES
where REGISTRATIONID in ({})
""".format(given_id_str)
#     print(user_apps_query)
user_apps_df_temp = pd.read_sql(user_apps_query, cursor)


user_Rake_query = """
SELECT CATEGORY,REGISTRATIONID,RAKE from MongoDB.analysis.DAILY_FORMATWISE_RAKE
where REGISTRATIONID in ({})
""".format(given_id_str)
user_Rake_df_temp = pd.read_sql(user_Rake_query, cursor)
user_Rake_df_temp = user_Rake_df_temp.loc[user_Rake_df_temp.groupby('REGISTRATIONID').RAKE.idxmax()].reset_index(drop=True)


Add_Cash_query = """
SELECT REGISTRATIONID,PAYAMOUNT,PAYDATE,PAYSTATUS,CREATED_DATE FROM TicTokdb.dbo.STBL_CASH_IN_WALLET
where REGISTRATIONID in ({}) AND PAYSTATUS = 'SUCCESS'
""".format(given_id_str)
Add_Cash_df_temp = pd.read_sql(Add_Cash_query, cursor)
Add_Cash_df_temp = Add_Cash_df_temp.loc[Add_Cash_df_temp.groupby('REGISTRATIONID').PAYDATE.idxmin()].reset_index(drop=True)

wb_bazzi_trans_query = """
SELECT REGISTRATIONID,CHALLENGE_ID,AMOUNT,TRANS_TYPE FROM TicTokdb.dbo.STBL_WB_BAAZI_TRANSACTION swbt
where REGISTRATIONID in ({})
""".format(given_id_str)
user_commongames_df_temp = pd.read_sql(wb_bazzi_trans_query, cursor)
#################################

user_loc_df_temp.set_index(['REGISTRATIONID'],inplace=True)
user_apps_df_temp.set_index(['REGISTRATIONID'],inplace=True)
Add_Cash_df_temp.set_index(['REGISTRATIONID'],inplace=True)
user_Rake_df_temp.set_index(['REGISTRATIONID'],inplace=True)
user_commongames_df_temp.set_index(['REGISTRATIONID'],inplace=True)
#     blocked_users_df_temp.set_index(['REGISTRATIONID'],inplace=True)

user_reg_df_temp.set_index(['REGISTRATIONID'],inplace=True)    
user_contacts_df_temp.set_index(['REGISTRATIONID'],inplace=True)

credit_df_temp = user_commongames_df_temp[user_commongames_df_temp.TRANS_TYPE == 'C']
debit_df_temp  = user_commongames_df_temp[user_commongames_df_temp.TRANS_TYPE == 'D']

#########################################
block_df_temp_list = []
count = 0
#     Name_similarity,matching_contact_count = 0,0
user = reg_id[0]
if count%25 == 0:
    print(user,count)
count+=1

p1_location = user_loc_df_temp[user_loc_df_temp.index==user]
if p1_location.shape[0] > 0:
    p1_lat, p1_lon = p1_location['LATITUDE'].iloc[0], p1_location['LONGITUDE'].iloc[0]
else:
    p1_lat, p1_lon = 0, 0
try:
    Rake_value_p1 = user_Rake_df_temp[user_Rake_df_temp.index==user]['RAKE'].iloc[0]
except:
    Rake_value_p1 = np.nan
try:
    Frst_AddCash_p1 = Add_Cash_df_temp[Add_Cash_df_temp.index==user]['PAYAMOUNT'].iloc[0]
except:
    Frst_AddCash_p1  = 0
# ###        
try:
    p1_name   = user_reg_df_temp[user_reg_df_temp.index==user]['NAME'].iloc[0]
except:
    p1_name = ''
        
p1_contact  =  user_contacts_df_temp[user_contacts_df_temp.index==user]['NAME']
# ###
p1_app    = user_apps_df_temp[user_apps_df_temp.index==user]['APP_NAME_ID']
p1_game   = set(user_commongames_df_temp[user_commongames_df_temp.index==user]['CHALLENGE_ID'])

#############################    
for user2 in given_id:
    if (user != user2):

        p2_location = user_loc_df_temp[user_loc_df_temp.index==user2]
        if p2_location.shape[0] > 0:
            p2_lat, p2_lon = p2_location['LATITUDE'].iloc[0], p2_location['LONGITUDE'].iloc[0]
        else:
            p2_lat, p2_lon = 0, 0 

        p1_p2_distance = np.nan
        if p1_location.shape[0] > 0 and p2_location.shape[0] > 0:
            p1_p2_distance = get_distance_between_locs(p1_lat, p1_lon, p2_lat, p2_lon)   ###
###                    
        try:
            p2_name   = user_reg_df_temp[user_reg_df_temp.index==user2]['NAME'].iloc[0]
        except:
            p2_name = ''

        Name_similarity = similarity(p1_name,p2_name)
        p2_contact  =  user_contacts_df_temp[user_contacts_df_temp.index==user2]['NAME']
        matching_contact_count = contact_overlap(p1_contact,p2_contact)
###
        try:
            Rake2 = user_Rake_df_temp[user_Rake_df_temp.index==user2]['RAKE'].iloc[0]
        except:
            Rake2 = np.nan
        try:
            Frst_AddCash2 = Add_Cash_df_temp[Add_Cash_df_temp.index==user2]['PAYAMOUNT'].iloc[0]
        except:
            Frst_AddCash2  = 0
        p2_app     = user_apps_df_temp[user_apps_df_temp.index==user2]['APP_NAME_ID']
        app_match  = app_overlap(p1_app,p2_app)

        p2_game = set(user_commongames_df_temp[user_commongames_df_temp.index==user2]['CHALLENGE_ID'])
        common_games = p1_game.intersection(p2_game)

        mean_Rake = (Rake_value_p1+Rake2)/2
        mean_frst_addcash = (Frst_AddCash_p1+Frst_AddCash2)/2

        if len(common_games) != 0 :

            common_games_percent_of_A_games = (len(common_games)/len(p1_game) )*100                    
            common_games_percent_of_B_games = ( len(common_games)/len(p2_game) )*100
            max_common_gameAB__of_A_B   = max(common_games_percent_of_A_games,common_games_percent_of_B_games)

            # common_games_A_B_profit   && # common_games_A_B_c_by_d
            com_game_credit_A = credit_df_temp[(credit_df_temp.index == user) & (credit_df_temp.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()
            com_game_debit_A  = debit_df_temp[(debit_df_temp.index == user) & (debit_df_temp.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()
            com_game_credit_B = credit_df_temp[(credit_df_temp.index == user2) & (credit_df_temp.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()
            com_game_debit_B  = debit_df_temp[(debit_df_temp.index == user2) & (debit_df_temp.CHALLENGE_ID.isin(common_games))]['AMOUNT'].sum()

            if ((com_game_debit_A+com_game_debit_B) != 0):
#                     A_B_Profit = ((com_game_credit_A+com_game_credit_B) - (com_game_debit_A+com_game_debit_B))/(com_game_debit_A+com_game_debit_B)
                A_B_C_BY_D = (com_game_credit_A+com_game_credit_B)/(com_game_debit_A+com_game_debit_B)
            else:
#                     A_B_Profit =  np.nan
                A_B_C_BY_D =  np.nan

            block_df_temp_list.append({
                    'P1_regID':user,
                    'P2_regID': user2,
                    'Name_similarity': Name_similarity,
                    'Contacts_Overlap_%':matching_contact_count,
                    'p1_p2_distance_KM' : p1_p2_distance,
                    'Apps_overlap_%': app_match,
                    'A_B_C_BY_D':A_B_C_BY_D,
                    'mean_Rake' : mean_Rake,
                    'mean_Frst_AddCash_1&2': mean_frst_addcash,
                    'max_common_gameAB_%_of_A&B' : max_common_gameAB__of_A_B,
#                         'A_B_Profit':A_B_Profit,



                })
        else:
            block_df_temp_list.append({
                'P1_regID':user,
                'P2_regID': user2,
                'Name_similarity': Name_similarity,
                'Contacts_Overlap_%':matching_contact_count,
                'p1_p2_distance_KM' : p1_p2_distance,
                'Apps_overlap_%': app_match,
                'mean_Rake' : mean_Rake,
                'mean_Frst_AddCash_1&2': mean_frst_addcash
            })

#     print('About to Complete')
Test_df = pd.DataFrame(block_df_temp_list)
# return Test_df